In [64]:
import pandas as pd
import numpy as np 
import math

# special
ALL = slice(None)

# number


def root_of(a: int | float, base: int = 2) -> int | float:
    return a**(1 / base)

# float


def round_to(a: float, places: int = 2) -> float:
    return round(a, places)


def floor(a: float) -> int:
    return math.floor(a)


def ceil(a: float) -> int:
    return math.ceil(a)

# string 


def concat(a: str, b: str) -> str:
    match (type(a), type(b)):
        case (pd.Series, str):
            return a.astype(str) + b
        case (str, pd.Series):
            return a + b.astype(str)
        case _:
            return a + b


def substr(a: str, start: int, end: int) -> str:
    if type(a) == pd.Series:
        return a.str.slice(start=start, stop=end)
    return a[start:end]


def strip(a: str, special_char: str | None = None) -> str:
    if type(a) == pd.Series:
        return a.str.strip(special_char)
    return a.strip(special_char)


def split(a: str, on_what: str, keep: int = 0) -> str:
    if type(a) == pd.Series:
        a =  a.str.split(on_what, expand = True)
        print(a)
        return a[keep]
    return a.split(on_what)[keep]


def replace(a: str, where: int, _with: str) -> str:
    if type(a) == pd.Series:
        a = a.str
    return a[:where] + _with + a[where + 1:]


def replace_all(a: str, what: str, _whith: str) -> str:
    if type(a) == pd.Series:
        a = a.str
    return a.replace(what, _whith)

# data conversion


def to_string(a: int | float) -> str:
    if type(a) == pd.Series:
        return a.astype(str)
    return str(a)


def to_int(a: str | float) -> int:
    if type(a) == pd.Series:
        return a.astype(int)
    return int(a)


def to_float(a: str | int) -> float:
    if type(a) == pd.Series:
        return a.astype(float)
    float(a)

# data reduction 


def get_sum(a) -> int | float:
    return np.sum(a)


def get_prod(a) -> int | float:
    return np.prod(a)


def get_min(a) -> int | float:
    return np.min(a)


def get_max(a) -> int | float:
    return np.max(a)


def get_mean(a) -> int | float:
    return np.mean(a)

# data evaluation 


def is_empty(a) -> bool:
    return pd.isnull(a)


def contains(a: str, c_str: str, at: int | None = None) -> bool:
    match (type(a), at):
        case (pd.Series, None):
            return a.str.contains(c_str)
        case (pd.Series, int):  # noqa: F841
            return a.str[at] == c_str
        case (str, int):  # noqa: F841
            return a[at] == c_str
        case _:
            return c_str in a


def starts_with(a: str, c_str: str) -> str:
    if type(a) == pd.Series:
        a = a.str
    return a.startswith(c_str)

In [65]:
expr_locals = {
    "root_of":root_of,
    "round_to":round_to,
    "floor":floor,
    "ceil":ceil,
    "concat":concat,
    "substr":substr,
    "strip":strip,
    'split':split,
    "replace":replace,
    "replace_all":replace_all,
    "to_string":to_string,
    "to_float":to_float,
    "to_int":to_int,
    "get_sum":get_sum,
    "get_prod":get_prod,
    "get_min":get_min,
    "get_max":get_max,
    "get_mean":get_mean,
    "is_empty":is_empty,
    "contains":contains,
    "starts_with":starts_with,
}
print(len(expr_locals.keys()))

21


### testing

In [93]:
df = pd.read_excel("alldtypes.xlsx")
df = df.replace({np.nan:None})
df

,INT,FLOAT,STRING,NONE,BOOL
0,1,1.1,text,None,True
1,2,2.1,text mit Spaces,None,False
2,3,3.3,"sumbole,%§/(",None,True


In [94]:
df = df.eval('`Purchasing Cost Center` = " "', engine= "python")
df

,INT,FLOAT,STRING,NONE,BOOL,BACKTICK_QUOTED_STRING_Purchasing_Cost_Center
0,1,1.1,text,None,True,
1,2,2.1,text mit Spaces,None,False,
2,3,3.3,"sumbole,%§/(",None,True,


In [96]:
df.eval('`test` = `Purchasing Cost Center` == " " ')

,INT,FLOAT,STRING,NONE,BOOL,BACKTICK_QUOTED_STRING_Purchasing_Cost_Center,test
0,1,1.1,text,None,True,,True
1,2,2.1,text mit Spaces,None,False,,True
2,3,3.3,"sumbole,%§/(",None,True,,True


In [60]:
df_1 = pd.read_excel("beispiel.xlsx", sheet_name="Tabelle1")
df_1

,Belegnummer,Belegdatum,Lieferant,Profitcenter
0,1,05.Jan,A,P1
1,2,06.Jan,B,P3
2,3,07.Jan,A,P1
3,5,09.Jan,B,P2
4,6,10.Jan,A,P3
5,7,11.Jan,A,P2


In [61]:
df_1.eval('@contains(`Profitcenter`, "P", 0)' , engine = "python")

0    True
1    True
2    True
3    True
4    True
5    True
Name: Profitcenter, dtype: bool

In [63]:
ALL = slice(None)
df.query("@ALL", engine="python")

AttributeError: 'Series' object has no attribute 'query'

In [ ]:
texte = """load data from excel file {file_name:string}( from the sheet {sheet_name:string|int = 0})( and set the index to the column {index:attribute|None = None})
save data to excel file {file_name:string}( in to the sheet {sheet_name:string = "Sheet1"})( and also keep the index)
select the rows where {query:expr}
select the columns named [{column:attribute}]*
rename column from {from_col:attribute} to {to_col:attribute}
delete the data( but keep the columns)
add row with the values [{value:dtype}]*
add row with the values [{column:attribute}={value:dtype}]*
delete row with the values [{value:dtype}]*
delete row with the values [{column:attribute}={value:dtype}]*
change row from [{org_value:dtype}]* to [{new_value:dtype}]*
change row from [{column:attribute}={org_value:dtype}]* to [{column:attribute}={new_value:dtype}]*
add column {column:attribute} with the value {value:expr}
change column {column:attribute} to the value {value:expr}
set column {column:attribute} to the value {value:expr}
delete column {column:attribute}
do nothing 
"""

texte = texte.replace("{", "\{")
texte = texte.replace("}","\}")
texte = texte.replace("_", "\_")
texte = texte.replace('"', "\dq")
texte = "\n".join([f"\subsubsection{{{t}}}" for t in texte.split("\n")])
print(texte)

\subsubsection{load data from excel file \{file\_name:string\}( from the sheet \{sheet\_name:string|int = 0\})( and set the index to the column \{index:attribute|None = None\})}
\subsubsection{save data to excel file \{file\_name:string\}( in to the sheet \{sheet\_name:string = \dqSheet1\dq\})( and also keep the index)}
\subsubsection{select the rows where \{query:expr\}}
\subsubsection{select the columns named [\{column:attribute\}]*}
\subsubsection{rename column from \{from\_col:attribute\} to \{to\_col:attribute\}}
\subsubsection{delete the data( but keep the columns)}
\subsubsection{add row with the values [\{value:dtype\}]*}
\subsubsection{add row with the values [\{column:attribute\}=\{value:dtype\}]*}
\subsubsection{delete row with the values [\{value:dtype\}]*}
\subsubsection{delete row with the values [\{column:attribute\}=\{value:dtype\}]*}
\subsubsection{change row from [\{org\_value:dtype\}]* to [\{new\_value:dtype\}]*}
\subsubsection{change row from [\{column:attribute\}=\

In [ ]:
test = "T392356-E-NW-VTV introduction in VF-DE"
substr(test,1,7)

'392356'

In [ ]:
test = "DE04M02240 b940650d-97fe-4a80-b01f-b2f3f1716ed4"
split(test, " ")

'DE04M02240'